<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a9be3d7e6d41d505a2079303ab94c1c7b48425447692c8360ff13be074206911
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-08 12:46:49
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.57 C
-------------------
Today PnL: -1.76 L (-1.11%)
Current PnL: -26.63 L (-16.98%)
CY Booked + Current PnL: -12.34 L (-7.87%)
-------------------
Total profit:  1.89 L
Total loss:  -28.52 L
-------------------
Total Booked + Current PnL: 14.64 L (11.27%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.13%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.52 C
Est FTT PnL: 95.65 L (61.11%)
Deployed:  1.30 C
Current:  1.57 C
CAGR/XIRR %: 10.38%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.91,14.85,5.70,21.40,17377.0,39429.0,304855.0,6.58,62.0,X-LC,5.52,14.0,2.27,1.95,22.12,XY25,NTT,FMCG
80,TTKPRESTIG,770.00,-0.41,-20.93,26.58,0.09,21181.0,-21090.0,79687.0,85.06,37.0,M-SC,3.22,253.0,-1.00,0.51,4.69,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-1.24,4.83,13.73,19.23,21656.0,7270.0,157731.0,22.81,75.0,L-LC,6.59,182.0,0.34,1.01,27.14,XY25,ATH,MINING
39,INDIGOPNTS,1408.00,0.59,-12.69,14.56,0.02,22191.0,-22151.0,152408.0,132.70,59.0,M-SC,7.81,240.0,-1.00,0.97,32.57,OX40N,NTT,PAINTS
49,MASFIN,398.61,-3.86,-1.35,23.72,22.05,22928.0,-1320.0,96660.0,-15.09,51.0,H-SC,6.61,168.0,-0.06,0.62,40.50,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,4546.37,5.51,-30.88,180.12,93.63,125704.0,-31174.0,69789.0,-25.90,51.0,H-SC,16.52,143.0,-0.25,0.45,5.51,SR,ATH,CHEMICALS
79,TRIDENT,48.00,4.69,-26.45,74.74,28.51,50712.0,-24404.0,67851.0,-13.08,54.0,M-SC,7.12,226.0,-0.48,0.43,16.10,XR,NTT,TEXTILES
37,IEX,219.00,4.11,5.04,41.49,48.63,87144.0,10082.0,210036.0,-30.08,71.0,H-SC,13.83,137.0,0.12,1.34,17.25,XR,NTT,MISC
22,DIXON,18931.72,2.12,-16.67,57.51,31.26,110594.0,-38464.0,192304.0,-61.06,35.0,X-MC,18.43,56.0,-0.35,1.23,2.50,X40N,ATH,IT
71,STARHEALTH,761.00,1.34,-12.82,68.92,47.26,168904.0,-36051.0,245072.0,19.04,42.0,H-SC,7.72,174.0,-0.21,1.57,31.30,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,MASFIN,398.61,-3.86,-1.35,23.72,22.05,22928.0,-1320.0,96660.0,-15.09,51.0,H-SC,6.61,168.0,-0.06,0.62,40.50,XR,ATH,FINANCE
56,RAJOOENG,143.10,-3.71,-29.52,98.09,39.61,70860.0,-30260.0,72240.0,-52.38,56.0,H-SC,31.23,136.0,-0.43,0.46,16.93,AR,ATH,MISC
82,VAIBHAVGBL,521.00,-2.97,-29.26,127.25,60.77,164538.0,-53472.0,129303.0,49.31,42.0,H-SC,4.47,158.0,-0.32,0.83,16.45,XR,NTT,JEWELLERY
35,ICICIGI,2252.93,-2.91,1.12,18.02,19.34,36462.0,2234.0,202343.0,-20.26,37.0,X-MC,3.63,71.0,0.06,1.29,17.23,X40,ATH,INSURANCE
57,RECLTD,446.00,-2.78,1.39,19.08,20.74,39305.0,2832.0,206002.0,33.49,62.0,M-MC,5.89,189.0,0.07,1.32,12.16,XY25,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.43,-1.31,110.67,107.91,161257.0,-1934.0,145710.0,-19.74,55.0,M-SC,10.18,220.0,-0.01,0.93,8.64,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.59,-12.69,14.56,0.02,22191.0,-22151.0,152408.0,132.70,59.0,M-SC,7.81,240.0,-1.00,0.97,32.57,OX40N,NTT,PAINTS
75,TATAELXSI,9161.00,-2.21,-17.51,60.06,32.03,65313.0,-23089.0,108746.0,-10.66,66.0,H-SC,6.96,157.0,-0.35,0.69,21.81,OX40N,NTT,IT
80,TTKPRESTIG,770.00,-0.41,-20.93,26.58,0.09,21181.0,-21090.0,79687.0,85.06,37.0,M-SC,3.22,253.0,-1.00,0.51,4.69,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.16,-21.53,44.61,13.47,94394.0,-58068.0,211599.0,-67.73,51.0,H-SC,4.78,159.0,-0.62,1.35,7.59,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,-2.78,1.39,19.08,20.74,39305.0,2832.0,206002.0,33.49,62.0,M-MC,5.89,189.0,0.07,1.32,12.16,XY25,NTT,FINANCE
17,COALINDIA,484.83,-1.24,4.83,13.73,19.23,21656.0,7270.0,157731.0,22.81,75.0,L-LC,6.59,182.0,0.34,1.01,27.14,XY25,ATH,MINING
87,WIPRO,420.00,-2.38,8.42,58.88,72.27,108176.0,14275.0,183723.0,-6.99,53.0,M-LC,6.18,101.0,0.13,1.17,15.12,XR,NTT,IT
37,IEX,219.00,4.11,5.04,41.49,48.63,87144.0,10082.0,210036.0,-30.08,71.0,H-SC,13.83,137.0,0.12,1.34,17.25,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,WIPRO,420.00,-2.38,8.42,58.88,72.27,108176.0,14275.0,183723.0,-6.99,53.0,M-LC,6.18,101.0,0.13,1.17,15.12,XR,NTT,IT
37,IEX,219.00,4.11,5.04,41.49,48.63,87144.0,10082.0,210036.0,-30.08,71.0,H-SC,13.83,137.0,0.12,1.34,17.25,XR,NTT,MISC
25,FINCABLES,1641.55,-0.43,-1.31,110.67,107.91,161257.0,-1934.0,145710.0,-19.74,55.0,M-SC,10.18,220.0,-0.01,0.93,8.64,OX40N,ATH,CABLES
49,MASFIN,398.61,-3.86,-1.35,23.72,22.05,22928.0,-1320.0,96660.0,-15.09,51.0,H-SC,6.61,168.0,-0.06,0.62,40.50,XR,ATH,FINANCE
88,ZYDUSLIFE,1286.17,-0.36,-2.54,39.17,35.63,80004.0,-5326.0,204248.0,-16.30,50.0,H-MC,2.92,119.0,-0.07,1.31,13.71,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.38,-16.68,32.96,10.78,65996.0,-40087.0,200231.0,-50.55,13.0,X-LC,5.78,1.0,-0.61,1.28,0.0,X40,NTT,FMCG
7,AWL,485.00,-0.28,-25.98,112.36,57.19,271246.0,-84719.0,241408.0,-65.03,27.0,X-MC,8.80,58.0,-0.31,1.54,0.0,XY24,NTT,FMCG
52,PAGEIND,51605.07,-1.27,-11.59,47.74,30.62,100053.0,-27474.0,209580.0,-37.21,29.0,X-MC,16.16,57.0,-0.27,1.34,0.0,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.18,-40.33,126.61,35.20,97200.0,-51899.0,76771.0,0.65,32.0,X-SC,17.34,93.0,-0.53,0.49,0.0,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-1.75,-22.53,82.66,41.51,168651.0,-59335.0,204030.0,-7.63,32.0,X-MC,8.98,65.0,-0.35,1.30,0.0,XY25,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UNITDSPR,1644.00,-2.30,-1.12,22.13,20.77,51237.0,-2615.0,231529.0,-9.72,33.0,X-MC,0.98,70.0,-0.05,1.48,5.72,X40N,NTT,BREWERIES
68,SIEMENS,4671.50,-1.74,-17.27,51.72,25.51,79623.0,-32145.0,153950.0,-5.41,44.0,H-LC,1.56,50.0,-0.40,0.98,13.45,AR,ATH,ELECTRICAL
20,DABUR,735.00,-0.46,1.37,41.76,43.70,105015.0,3404.0,251472.0,-7.04,63.0,X-MC,2.09,73.0,0.03,1.61,16.63,XY24,BTT,FMCG
84,VBL,671.64,-1.82,1.08,34.22,35.67,109249.0,3413.0,319255.0,-10.92,62.0,X-LC,2.19,5.0,0.03,2.04,15.08,X40N,ATH,FMCG
45,JIOFIN,387.00,-2.62,-4.93,30.94,24.48,78641.0,-13192.0,254173.0,-6.53,46.0,X-LC,2.20,37.0,-0.17,1.62,47.33,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.28,-25.98,112.36,57.19,271246.0,-84719.0,241408.0,-65.03,27.0,X-MC,8.80,58.0,-0.31,1.54,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-1.18,-40.33,126.61,35.20,97200.0,-51899.0,76771.0,0.65,32.0,X-SC,17.34,93.0,-0.53,0.49,0.0,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-1.75,-22.53,82.66,41.51,168651.0,-59335.0,204030.0,-7.63,32.0,X-MC,8.98,65.0,-0.35,1.30,0.0,XY25,ATH,FMCG
43,ITC,452.00,-0.38,-16.68,32.96,10.78,65996.0,-40087.0,200231.0,-50.55,13.0,X-LC,5.78,1.0,-0.61,1.28,0.0,X40,NTT,FMCG
52,PAGEIND,51605.07,-1.27,-11.59,47.74,30.62,100053.0,-27474.0,209580.0,-37.21,29.0,X-MC,16.16,57.0,-0.27,1.34,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.28,-26.49,94.31,42.84,45069.0,-17218.0,47788.0,-52.07,56.0,X-SC,27.85,83.0,-0.38,0.31,8.12,XY24,NTT,MISC
58,RELAXO,1176.00,-0.52,-48.22,194.15,52.31,145927.0,-69998.0,75162.0,-44.83,41.0,X-SC,7.74,92.0,-0.48,0.48,0.61,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.18,-40.33,126.61,35.20,97200.0,-51899.0,76771.0,0.65,32.0,X-SC,17.34,93.0,-0.53,0.49,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.62,1.73,21.96,24.08,28361.0,2200.0,129150.0,-6.45,55.0,X-SC,6.41,91.0,0.08,0.83,21.69,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.69,-16.83,72.32,43.32,97965.0,-27412.0,135460.0,-28.55,48.0,X-SC,7.81,90.0,-0.28,0.87,4.01,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.38,-16.68,32.96,10.78,65996.0,-40087.0,200231.0,-50.55,13.0,X-LC,5.78,1.0,-0.61,1.28,0.00,X40,NTT,FMCG
78,TMPV,600.00,-0.94,-26.22,66.69,22.98,111144.0,-59241.0,166657.0,-23.08,47.0,X-LC,4.49,2.0,-0.53,1.06,4.82,XY24,NTT,AUTO
76,TCS,4389.97,-2.20,-11.38,36.21,20.71,121373.0,-43045.0,335192.0,-23.62,49.0,X-LC,2.61,3.0,-0.35,2.14,12.00,X40,ATH,IT
84,VBL,671.64,-1.82,1.08,34.22,35.67,109249.0,3413.0,319255.0,-10.92,62.0,X-LC,2.19,5.0,0.03,2.04,15.08,X40N,ATH,FMCG
41,INFY,1972.00,-1.35,10.07,21.96,34.24,75630.0,31512.0,344400.0,-14.21,51.0,X-LC,7.63,6.0,0.42,2.20,19.19,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-2.04,-35.07,104.65,32.89,52465.0,-27072.0,50134.0,-719.08,56.0,L-MC,7.60,259.0,-0.52,0.32,38.07,XR,NTT,BANKS
49,MASFIN,398.61,-3.86,-1.35,23.72,22.05,22928.0,-1320.0,96660.0,-15.09,51.0,H-SC,6.61,168.0,-0.06,0.62,40.50,XR,ATH,FINANCE
13,BSOFT,831.70,-2.41,-21.55,92.21,50.79,101743.0,-30315.0,110338.0,0.50,53.0,H-SC,10.72,171.0,-0.30,0.71,28.47,XR,ATH,IT
72,SURYODAY,216.00,-0.38,-18.16,50.34,23.03,73770.0,-32528.0,146543.0,60.09,57.0,H-SC,5.05,165.0,-0.44,0.94,45.40,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.59,-12.69,14.56,0.02,22191.0,-22151.0,152408.0,132.70,59.0,M-SC,7.81,240.0,-1.00,0.97,32.57,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-2.21,-28.36,57.61,12.90,90171.0,-61976.0,156520.0,130.77,66.0,H-SC,10.51,173.0,-0.69,1.00,30.15,OX40N,NTT,FINANCE
75,TATAELXSI,9161.00,-2.21,-17.51,60.06,32.03,65313.0,-23089.0,108746.0,-10.66,66.0,H-SC,6.96,157.0,-0.35,0.69,21.81,OX40N,NTT,IT
17,COALINDIA,484.83,-1.24,4.83,13.73,19.23,21656.0,7270.0,157731.0,22.81,75.0,L-LC,6.59,182.0,0.34,1.01,27.14,XY25,ATH,MINING
85,VOLTAS,1856.00,-2.17,15.38,25.84,45.20,57167.0,29493.0,221235.0,4.17,64.0,X-MC,4.59,80.0,0.52,1.41,23.22,XY25,NTT,AC
36,ICICIPRULI,790.00,-0.63,12.57,16.13,30.73,35773.0,24773.0,221778.0,-13.70,69.0,X-MC,6.46,79.0,0.69,1.42,26.95,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,21.70
1,25,44.74
2,50,75.48


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.33
LC    32.60
MC    28.10
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     24.20
X40      21.73
XY25     18.24
X40N     11.73
XR        8.35
AR        7.89
OX40N     6.99
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    29.44
X-MC    22.15
H-SC    21.05
M-SC    10.36
X-SC     7.20
H-MC     4.31
M-MC     1.32
M-LC     1.17
L-LC     1.01
H-LC     0.98
L-SC     0.72
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.53,-8.37,43.93
IT,11.72,-16.57,75.37
FINANCE,10.18,-12.01,59.02
REFINERIES,9.72,-5.75,32.50
MISC,6.62,-25.33,76.60
ELECTRICAL,5.43,-10.58,51.08
PAINTS,5.22,-12.91,29.68
INSURANCE,4.28,-1.35,36.03
PHARMA,3.64,-3.24,35.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3187431.0,21
AR,1337844.0,10
XR,1185836.0,12
X40,1120058.0,15
X40N,907334.0,9
XY25,850899.0,10
OX40N,694804.0,10
SR,280664.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3502484.0,24
X-MC,1566630.0,16
M-SC,1438497.0,15
X-LC,1378627.0,16
X-SC,794742.0,8
H-MC,405508.0,3
L-SC,177157.0,2
M-LC,108176.0,1
H-LC,79623.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1241409.0      6
           AR         917674.0      5
M-SC       XY24       829002.0      6
H-SC       XR         759501.0      7
X-LC       XY25       564120.0      6
X-MC       X40        505756.0      7
           XY24       442280.0      3
X-LC       X40        420295.0      6
X-MC       X40N       392776.0      4
X-SC       X40N       310131.0      3
H-SC       OX40N      303236.0      4
M-SC       OX40N      300409.0      5
X-SC       XY24       290604.0      3
H-SC       SR         280664.0      2
X-MC       XY25       225818.0      2
H-MC       AR         211157.0      2
X-LC       X40N       204427.0      2
H-MC       XY24       194351.0      1
X-SC       X40        194007.0      2
X-LC       XY24       189785.0      2
M-SC       XR         179696.0      2
           AR         129390.0      2
M-LC       XR         108176.0      1
L-SC       OX40N       91159.0      1
           XR          85998.0      1
H-LC       AR          79623.0      1
L-MC       XR          52465.0      1
M-MC       XY25        39305.0      1
L-LC       XY25        21656.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
